In [1]:
import sklearn
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import re
import pickle
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = sqlite3.connect('../db.sqlite3')

In [10]:
df_raw = pd.read_sql_query("SELECT * FROM scraper_auto WHERE bron='marktplaats'", conn)

In [11]:
target_names = ["bouwjaar", "kilometer_stand", "vermogen", "prijs", "is_handgeschakeld", "is_benzine", "upload_datum", "apk"]
df = df_raw[target_names]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,477.000000,476.000000,477.000000,466.000000,475.000000,477.000000
mean,2007.863732,179284.502101,80.874214,4753.351931,0.957895,0.727463
std,1.763390,58676.832720,13.944251,46213.094067,0.201041,0.445732
min,2005.000000,15570.000000,51.000000,1.000000,0.000000,0.000000
25%,2006.000000,138128.500000,77.000000,1950.000000,1.000000,0.000000
50%,2007.000000,177559.000000,78.000000,2562.500000,1.000000,1.000000
75%,2009.000000,212451.000000,84.000000,3250.000000,1.000000,1.000000
max,2014.000000,368884.000000,155.000000,999999.000000,1.000000,1.000000


In [12]:
MAX_PRICE = 30000
MIN_PRICE = 1000
df = df[(df['prijs'] > MIN_PRICE) & (df['prijs'] < MAX_PRICE)]

MIN_BOUWJAAR = 2000
df = df[df['bouwjaar'] > MIN_BOUWJAAR]

MAX_VERMOGEN = 500
df = df[df['vermogen'] < MAX_VERMOGEN]

df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,441.000000,440.000000,441.000000,441.00000,439.000000,441.000000
mean,2007.950113,175042.979545,80.446712,2717.76644,0.956720,0.736961
std,1.777960,55103.747892,13.221797,873.78392,0.203719,0.440783
min,2005.000000,15570.000000,51.000000,1150.00000,0.000000,0.000000
25%,2006.000000,137210.250000,77.000000,1999.00000,1.000000,0.000000
50%,2007.000000,176454.000000,78.000000,2650.00000,1.000000,1.000000
75%,2009.000000,207621.750000,84.000000,3250.00000,1.000000,1.000000
max,2014.000000,368884.000000,155.000000,6950.00000,1.000000,1.000000


In [13]:
df['upload_datum'] = pd.to_datetime(df['upload_datum'])
df['apk'] = pd.to_datetime(df['apk'])

average_upload_datum = df['upload_datum'].mean().toordinal()
average_apk =df['apk'].mean().toordinal()

def upload_datum_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_upload_datum)
    else:
        return abs(datetime.now().toordinal() - date.toordinal())
    
def apk_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_apk)
    else:  
        return abs(datetime.now().toordinal() - date.toordinal())

df['upload_datum'] = df['upload_datum'].apply(upload_datum_to_ordinal).astype(int)
df['apk'] = df['apk'].apply(apk_to_ordinal).astype(int)

# def parse_titel(titel):
#     cleaned = re.sub('[^a-zA-Z]+', ' ', titel.lower())
#     stripped = line = re.sub(r'\b\w{1,3}\b', '', cleaned)
    
#     return stripped

# df['titel'] = df['titel'].apply(parse_titel)

In [14]:
df = df.dropna()
#df.isna().sum()

In [15]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [16]:
df.to_pickle("./data.pkl")